In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
! pip install glove-python-binary

     |████████████████████████████████| 948 kB 4.5 MB/s 


In [3]:
from glove import Glove
glove=Glove()
glove_model=glove.load("/content/gdrive/MyDrive/FYP/CAA_NRC/Glove/glove.model")

In [5]:
import pandas as pd
import numpy as np
df=pd.read_csv("/content/gdrive/MyDrive/FYP/CAA_NRC/CAA_NRC-preprocessed.csv")
y=df["label"]


In [6]:
maxi=-1
for i,rev in enumerate(df['new_text']):
  tokens=rev.split()
  if(len(tokens)>maxi):
    maxi=len(tokens)
print(maxi)

61


In [7]:
import tensorflow as tf
maxlen=66
max_words=len(glove_model.dictionary)
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words)
def padded_sequence(X):
  tokenizer.fit_on_texts(X)
  sequences = tokenizer.texts_to_sequences(X)
  data = pad_sequences(sequences, maxlen=maxlen)
  return data
X=padded_sequence(df["new_text"])

In [8]:
import numpy as np
embed_matrix=np.zeros(shape=(max_words,300))
d={}
for word,i in tokenizer.word_index.items():
  d[i]=word

In [9]:
for i in X:
  for j in range(len(i)):
    if i[j]==0:
      pass
    else:
      p=d[i[j]]
      if p in glove_model.dictionary:
        i[j]=sum(glove_model.word_vectors[glove_model.dictionary[p]])
      else:
        i[j]=0
  print(i)


Streaming output truncated to the last 5000 lines.
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 1 1 0 1 0 0 2 2 1
 2 1 2 0 0 0 1 1 1 0 1 1 0 1 0 1 1 0 2 2 0 0 2 0 0 0 0 1 2]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 2 2 1 1 0 1 0 2 0 1 0 1 0 1 0 2 0 0
 1 0 0 0 1 0 0 2 1 0 1 0 2 0 1 0 0 1 2 0 1 0 1 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 3 0 0 0 2 1 0 2 0 0 1 2 0 1 1 0 0 1 1 0 0 2 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 2 0 1 0 1 0 0 0 0 0 2 3 0 0 0 0 1 2 2 2 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 0
 0 0 0 0 2 0 0 2 2 0 2 2 0 2 0 1 0 0 0 0 2 0 0 0 2 0 1 1 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 2 0 0 1 0 0 0 0 0 1 0
 0 2 0 1 0 2 2 2 1 1 0 2 0 0 0 0 1 2 0 0 0 0 1 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 2 0 0 2 1 3 0 0
 2 2 2 1 0 0 0 0 0 2 0 0 0 0 2 1 1 2 2 0 0 3 0 0 2 1 3 0 0]
[0 0

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [11]:
#Metric calculation function
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
def display_metrics(y_test,y_pred):
    print(accuracy_score(y_test, y_pred))
    print(f1_score(y_test, y_pred, average="macro"))
    print(precision_score(y_test, y_pred, average="macro"))
    print(recall_score(y_test, y_pred, average="macro"))

In [12]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=10000).fit(X_train, y_train)
y_pred=clf.predict(X_test)

In [13]:
display_metrics(y_test,y_pred)

0.42396907216494845
0.3119909646649021
0.37190741448463016
0.34089802293607946


In [14]:
from sklearn import svm 
clf = svm.SVC(kernel='linear',verbose=1,max_iter=10000)
clf.fit(X_train, y_train)

[LibSVM]

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


SVC(kernel='linear', max_iter=10000, verbose=1)

In [15]:
y_pred=clf.predict(X_test)
display_metrics(y_test,y_pred)

0.4252577319587629
0.26036757301107755
0.24715073529411766
0.3240299596531891


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
from sklearn import tree
DTclf = tree.DecisionTreeClassifier()
DTclf.fit(X_train, y_train)

DecisionTreeClassifier()

In [17]:
y_pred=DTclf.predict(X_test)
display_metrics(y_test,y_pred)

0.4497422680412371
0.423272798752675
0.4228822372220186
0.42372735857155125
